In [1]:
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler


sns.set(rc={'figure.figsize':(10,5)})
sns.set_theme(style="whitegrid")

In [2]:
df = pd.read_csv("data/genres_v2.csv", low_memory=False)
print(df.columns)
df.head()

FileNotFoundError: ignored

In [ ]:
df['duration_ms'].value_counts()

Algumas das features não tem significado nenhum na hora de tentarmos prever o gênero da música (como _ID_, _URI_, duração, etc), sendo assim, iremos tirá-los do dataset para continuarmos a análise.

In [ ]:
df.drop(['type', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms', 'song_name', 'Unnamed: 0', 'title'], axis=1, inplace=True)
df.head()

Apenas para contexo, uma explicação melhor de cada uma dessas features pode ser encontrada [aqui](https://developer.spotify.com/documentation/web-api/reference/#/operations/get-audio-features).  

#### **Tratamento de dados inválidos**


- Procurando por *NaN* e *None*

In [3]:
df.isnull().sum().sort_values(ascending=False)

NameError: ignored

Como não há valores nulos, podemos prosseguir.

- Procurando por valores divergentes do esperado pela documentação:

In [ ]:
count = df.shape[0]
df.drop(df[(df['danceability'] > 1) | (df['danceability'] < 0)].index, inplace=True)
print("Dropped by danceability: ", count - df.shape[0])

In [ ]:
count = df.shape[0]
df.drop(df[(df['energy'] > 1) | (df['energy'] < 0)].index, inplace=True)
print("Dropped by energy: ", count - df.shape[0])

In [ ]:
count = df.shape[0]
df.drop(df[(df['key'] > 11) | (df['key'] < -1)].index, inplace=True)
print("Dropped by key: ", count - df.shape[0])

In [ ]:
count = df.shape[0]
df.drop(df[(df['loudness'] > 0) | (df['loudness'] < -60)].index, inplace=True)
print("Dropped by loudness: ", count - df.shape[0])

In [ ]:
count = df.shape[0]
df.drop(df[(df['mode'] != 1) & (df['mode'] != 0)].index, inplace=True)
print("Dropped by mode: ", count - df.shape[0])

In [ ]:
count = df.shape[0]
df.drop(df[(df['speechiness'] > 1) | (df['speechiness'] < 0)].index, inplace=True)
print("Dropped by speechiness: ", count - df.shape[0])

In [ ]:
count = df.shape[0]
df.drop(df[(df['acousticness'] > 1) | (df['acousticness'] < 0)].index, inplace=True)
print("Dropped by acousticness: ", count - df.shape[0])

In [ ]:
count = df.shape[0]
df.drop(df[(df['instrumentalness'] > 1) | (df['instrumentalness'] < 0)].index, inplace=True)
print("Dropped by instrumentalness: ", count - df.shape[0])

In [ ]:
count = df.shape[0]
df.drop(df[(df['liveness'] > 1) | (df['liveness'] < 0)].index, inplace=True)
print("Dropped by liveness: ", count - df.shape[0])

In [ ]:
count = df.shape[0]
df.drop(df[(df['valence'] > 1) | (df['valence'] < 0)].index, inplace=True)
print("Dropped by valence: ", count - df.shape[0])

Como é possível ver, os dados estão de acordo com a documentação e completos, sendo assim podemos realizar a analise deles.

#### **Análise Inicial dos Dados**

- Quantidade de músicas por gênero:

In [ ]:
mean_count = df.groupby('genre').count().mean()[0]

f, ax = plt.subplots(1, 1, figsize=(30, 6))
sns.histplot(df, x='genre', ax=ax)
ax.axhline(mean_count, linestyle='dashed', label='mean', color='red')
ax.legend()

Como podemos ver, o dataset está bem desbalanceado. Para balancear vamos fazer duas coisas:
* Retirar o genero 'Pop', já que o mesmo possui poucos dados quanto comparado ao resto.
* Diminuir o número de músicas do DarkTrap e Undergroup Rap para o valor médio de músicas por gênero.

In [ ]:
df.drop(df[df['genre'] == 'Pop' ].index, inplace=True)

dt_count = df[df['genre'] == 'Dark Trap'].shape[0]
df.drop(df[df['genre'] == 'Dark Trap'].sample(int(dt_count - mean_count)).index, inplace=True)

ur_count = df[df['genre'] == 'Underground Rap'].shape[0]
df.drop(df[df['genre'] == 'Underground Rap'].sample(int(ur_count - mean_count)).index, inplace=True)

Desse modo, teremos:

In [ ]:
mean_count = df.groupby('genre').count().mean()[0]

f, ax = plt.subplots(1, 1, figsize=(30, 6))
sns.histplot(df, x='genre', ax=ax)
ax.axhline(mean_count, linestyle='dashed', label='new mean', color='red')
ax.legend()

Agora que está mais balanceado, iremos verificar a correlação entre as variáveis que utilizaremos para determinar o gênero:

In [ ]:
corr = df.corr()

plt.figure(figsize=(10, 8))
plt.imshow(corr, cmap='Blues', interpolation='none', aspect='auto')
plt.colorbar()
plt.xticks(range(len(corr)), corr.columns, rotation='vertical')
plt.yticks(range(len(corr)), corr.columns);
plt.suptitle('Correlation between variables', fontsize=15, fontweight='bold')
plt.grid(False)
plt.show()

É possível ver que, no geral, as features são pouco correlacionadas. Alguns pares, como *loudness-energy* e *valence-danceability* são mais interligados, porém não são correlacionados o suficiente para podermos simplesmente ignorar um dos elemento de cada par.

#### **Normalizando os dados**

In [ ]:
X = df.iloc[:, :-1]
y = df.iloc[:, -1]

scaler = StandardScaler()
X = scaler.fit_transform(X)

#### **Análise e Redução da Dimensionalidade**

In [ ]:
pca = PCA()
pca.fit(X)

exp_var_cumul = {   'Número de Componentes': range(1, X.shape[1] + 1),
                    'Variância Explicada': np.cumsum(pca.explained_variance_ratio_)
                }


f, ax = plt.subplots(1, 1)

plt.xlabel('Número de Componentes')
plt.ylabel('Variância Explicada Acumulada')

ax.fill_between(exp_var_cumul['Número de Componentes'], exp_var_cumul['Variância Explicada'])
ax.plot(exp_var_cumul['Número de Componentes'], exp_var_cumul['Variância Explicada'], 'ro--')

Utilizando o PCA para visualizar a classificação com 2 componentes


In [ ]:
pca = PCA(n_components=2)
Xt = pca.fit_transform(X)


# Adicionando uma columna novo onde as classes se tornam numéricas
df['num_genre'] = 0
uniq_genres = df['genre'].unique()
i = 0
for genre in uniq_genres:
    df['num_genre'] = np.where(df["genre"] == genre, i, df['num_genre'])
    i += 1

plt.scatter(Xt[:,0], Xt[:,1], c=df['num_genre'], alpha = 1)

In [ ]:
for label in uniq_genres:
    plt.scatter(Xt[y==label, 0], Xt[y==label, 1], label=label)

plt.legend()
plt.show()

O resultado da visualização **não** ficou muito satisfatório, portanto iremos testar com T-SNE

T-SNE (t-distributed Stochastic Neighbor Embedding) é um algoritmo de visualização de dados usado para reduzir a dimensionalidade de um conjunto de dados com alta dimensionalidade para que possa ser visualizado em um gráfico bidimensional. É particularmente útil para visualizar clusters ou padrões em conjuntos de dados de alta dimensionalidade, como imagens ou texto.

O T-SNE funciona comparando as distâncias entre os pontos de dados no espaço de alta dimensionalidade e, em seguida, mapeando essas distâncias para o espaço de baixa dimensionalidade de maneira a manter a relação de distância o máximo possível. Isso significa que os pontos de dados que estão próximos uns dos outros no espaço de alta dimensionalidade serão colocados próximos uns dos outros no gráfico de visualização, enquanto os pontos de dados que estão distantes uns dos outros no espaço de alta dimensionalidade serão colocados distantes uns dos outros no gráfico de visualização.

Um dos principais benefícios do T-SNE é que ele é muito bom em preservar a estrutura dos dados, o que significa que os padrões e clusters presentes nos dados de alta dimensionalidade são geralmente visíveis no gráfico de visualização. No entanto, o T-SNE também é conhecido por ser um pouco lento e por ter dificuldade em lidar com conjuntos de dados muito grandes.

A principal diferença entre T-SNE e PCA é como eles reduzem a dimensionalidade dos dados. O PCA é um algoritmo linear que procura encontrar as direções de maior variação nos dados e, em seguida, projeta os dados nessas direções. Isso significa que o PCA é mais adequado para conjuntos de dados que seguem uma distribuição linear e tem uma variação clara. Além disso, o PCA é muito rápido e pode lidar com conjuntos de dados muito grandes, mas é menos bom em preservar a estrutura dos dados.

In [ ]:
tsne = TSNE(n_components=2)
z = tsne.fit_transform(X)

In [ ]:
sns.scatterplot(x=z[:, 0], y=z[:, 1], hue=df.genre.tolist(),
                palette=sns.color_palette("hls", 14)).set(title="T-SNE Projection") 

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X, y)

Realizando One-Hot-Enconding do *target* para separar as diferentes classes, evitando, assim, a imposição de uma ordenação implícita.

In [ ]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore')
y_train = enc.fit_transform(y_train.to_numpy().reshape(-1, 1))
y_test = enc.fit_transform(y_test.to_numpy().reshape(-1, 1))

Optamos por usar técnicas de *boosting* para classificação, uma vez que se mostram muito eficientes em conjuntos de dados tabulares.

O boosting é um método de aprendizado de máquina que se baseia em criar um conjunto de classificadores fracos e, em seguida, combiná-los em um classificador forte. Os classificadores fracos são modelos que são ligeiramente melhores do que o acaso, mas ainda não são suficientemente precisos por si só. No entanto, quando combinados, os classificadores fracos podem produzir um classificador forte que é muito mais preciso.

O processo de boosting é realizado iterativamente. Em cada iteração, um novo classificador é treinado e adicionado ao conjunto de classificadores. O conjunto de classificadores é atualizado para dar mais peso aos exemplos que foram classificados incorretamente pelos classificadores anteriores, de modo que os novos classificadores possam se concentrar em corrigir esses erros. Isso permite que os classificadores sejam "apostos" uns nos outros, resultando em um classificador final mais preciso.

O boosting é amplamente utilizado em muitos tipos diferentes de algoritmos de aprendizado de máquina, incluindo árvores de decisão, regressão logística e muito mais. Alguns exemplos populares de algoritmos de boosting incluem o XGBoost e o AdaBoost.

XGBoost é uma implementação avançada de árvores de decisão, um tipo de algoritmo de aprendizado de máquina supervisionado. Ele funciona construindo uma série de árvores de decisão em um conjunto de dados de treinamento e, em seguida, as usa para prever valores em um conjunto de dados de teste. Cada árvore de decisão é criada a partir de uma amostra aleatória do conjunto de dados de treinamento e é treinada usando um algoritmo de otimização, como o gradiente aumentado. As árvores são então combinadas para formar o modelo final, que é usado para fazer previsões.

Uma das principais vantagens do XGBoost é que ele é muito eficiente e rápido no treinamento de modelos. Isso é devido ao uso de técnicas avançadas de otimização, como o gradiente aumentado, bem como à implementação deficiente em código nativo. Além disso, o XGBoost fornece uma série de opções de tunagem de modelo, como regularização e seleção de características, o que o torna um poderoso ferramenta para otimizar o desempenho do modelo.

In [ ]:
parameters = {
    'max_depth': range (2, 5, 1),
    'n_estimators': range(60, 140, 40),
    'learning_rate': [0.1, 0.01, 0.05]
}

In [ ]:
from xgboost import XGBClassifier

model = XGBClassifier(objective= 'binary:logistic', nthread=4)

In [ ]:
from sklearn.model_selection import GridSearchCV

grid_search = GridSearchCV(estimator=model, param_grid=parameters, scoring = 'roc_auc', n_jobs = 10, cv = 10, verbose=10)

In [ ]:
grid_search.fit(x_train, y_train.toarray())

In [ ]:
best_model = grid_search.best_estimator_

In [ ]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(best_model, x_train, y_train.toarray(), cv=5)
print("Acurácia média no Cross-Validation de treino: %.2f" % scores.mean())

In [ ]:
scores = cross_val_score(best_model, x_test, y_test.toarray(), cv=5)
print("Acurácia média no Cross-Validation de teste: %.2f" % scores.mean())